In [1]:
# import libraries

from utils import text_processing
from utils import get_weather
import datetime

import requests

import pandas as pd

from bs4 import BeautifulSoup

import re

import nltk
import os
import string
import numpy as np
import copy 
import pickle
import re
import math

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



import pandas as pd

import scipy
from scipy import sparse

import numpy as np
from sklearn.model_selection import TimeSeriesSplit

import pickle

In [2]:
# get article from yesterday

#https://understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-23-2023

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

yesterday_day = yesterday.day
yesterday_month = yesterday.month
yesterday_year = yesterday.year


In [3]:
file = text_processing.get_article_from_yesterday(yesterday_day,yesterday_month,yesterday_year)

In [4]:
data = text_processing.read_html(file)

In [5]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

In [6]:
def preprocess_all_text(data):
    pattern = "\[(\d+)\]"


    data['main_html_v1'] = data['main_html'].apply(lambda x: re.sub(pattern,"",str(x)))
    data['main_html_v2'] = data['main_html_v1'].apply(lambda x: re.sub(r'http(\S+.*\s)',"",x))
    data['main_html_v3'] = data['main_html_v2'].apply(lambda x: re.sub(r'2022|2023|©2022|©2023|\xa0|\n',"",x))

    data['main_html_v4'] = data['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)
    
    data['main_html_v5'] = data['main_html_v4'].apply(lambda x: text_processing.remove_names_and_dates(x))
    
    return data

In [7]:
data_preprocessed = preprocess_all_text(data)

In [8]:
#data_preprocessed.head()

In [9]:
data_preprocessed = data_preprocessed.drop(['main_html_v1','main_html_v2','main_html_v3','main_html_v4'],axis=1)

In [10]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikita_voitishyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
data_preprocessed['report_text_lemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"lemm"))
data_preprocessed['report_text_stemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"stemm"))

In [12]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...


In [13]:
docs = data_preprocessed['report_text_lemm'].tolist()

In [14]:
#tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
#cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

In [15]:
# CountVectorizer + tfidf:create a vecror with numbers

cv = CountVectorizer()
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

tf_idf_vector = tfidf_transformer.transform(word_count_vector)

#<340x7401 sparse matrix of type '<class 'numpy.float64'>'
#	with 213470 stored elements in Compressed Sparse Row format>

In [16]:
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [17]:
feature_names = cv.get_feature_names()
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [18]:
data_preprocessed['keywords'] = data_preprocessed['report_text_stemm'].apply(lambda x: text_processing.convert_doc_to_vector(x,feature_names,tf_idf_vector))

In [19]:
data_preprocessed['keywords'][0]

{'russian': 0.708,
 'forc': 0.275,
 'ukrainian': 0.245,
 'twenty': 0.168,
 'april': 0.153,
 'claim': 0.132,
 'militari': 0.127,
 'five': 0.122,
 'report': 0.107,
 'two': 0.092}

In [20]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':..."


#### Part of scropt: 3_final_preprocess

In [21]:
data_preprocessed["date_datetime"] = pd.to_datetime(data_preprocessed["date"])
data_preprocessed['date_tomorrow_datetime'] = data_preprocessed['date_datetime'].apply(lambda x: x+datetime.timedelta(days=1))
data_preprocessed = data_preprocessed.rename(columns = {"date_datetime":"report_date"})

In [22]:
# погода на всі регіони треба: модифікувати скрипт get_weather?

region = "Дніпропетровщина"
today_date = "2023-04-27"

weather_for_12_hours = get_weather.get_weather_for_12_hours(region,today_date)

Weather data for the 
region Дніпропетровщина; 
start_date 2023-04-27; 
end_date 2023-04-27; 
is ready


In [23]:
weather_for_12_hours.head()

,datetime,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,source
0,00:00:00,1682542800,55.1,55.1,63.76,43.1,0.000,9.5,0.0,0.0,...,15.0,91.2,0.0,NaN,0.0,10.0,Overcast,cloudy,None,fcst
1,01:00:00,1682546400,54.9,54.9,66.88,44.1,0.000,9.5,0.0,0.0,...,15.0,89.2,0.0,NaN,0.0,10.0,Partially cloudy,partly-cloudy-night,None,fcst
2,02:00:00,1682550000,54.6,54.6,71.58,45.6,0.000,9.5,0.0,0.0,...,6.9,80.0,0.0,NaN,0.0,10.0,Partially cloudy,partly-cloudy-night,None,fcst
3,03:00:00,1682553600,53.7,53.7,76.53,46.5,0.000,9.5,0.0,0.0,...,15.0,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,None,fcst
4,04:00:00,1682557200,53.5,53.5,76.51,46.3,0.004,14.3,0.0,0.0,...,14.5,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,None,fcst


In [24]:
weather_for_12_hours['date'] = today_date
weather_for_12_hours["day_datetime"] = pd.to_datetime(weather_for_12_hours["date"])

In [25]:
weather_for_12_hours.columns

Index(['datetime', 'datetimeEpoch', 'temp', 'feelslike', 'humidity', 'dew',
       'precip', 'precipprob', 'snow', 'snowdepth', 'preciptype', 'windgust',
       'windspeed', 'winddir', 'pressure', 'visibility', 'cloudcover',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions',
       'icon', 'stations', 'source', 'date', 'day_datetime'],
      dtype='object')

In [26]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords,report_date,date_tomorrow_datetime
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':...",2023-04-25,2023-04-26


In [27]:
#data_preprocesdf_isw_short = data_preprocessed[['date','date_tomorrow_datetime','keywords','report_text_lemm']]

In [28]:
data_vectorised = tf_idf_vector.toarray()
vectors_df = pd.DataFrame(data_vectorised)
vectors_df['date'] = pd.to_datetime(today)


In [29]:
vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,831,832,833,834,835,836,837,838,839,date
0,0.005094,0.005094,0.015281,0.005094,0.005094,0.005094,0.010188,0.005094,0.010188,0.005094,...,0.005094,0.030563,0.005094,0.015281,0.015281,0.005094,0.010188,0.005094,0.025469,2023-04-26


In [42]:
df_isw_short = data_preprocessed[['date','report_text_lemm','keywords','date_tomorrow_datetime']]

In [43]:
df_isw_short.head()

,date,report_text_lemm,keywords,date_tomorrow_datetime
0,2023-04-25,russian offens campaign ass april twenty five...,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':...",2023-04-26


In [44]:
tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

#Uncommit if you want to load exist matrix
tfidf_vector = scipy.sparse.load_npz('data/matrix/tfidf_vector_train.npz')

In [32]:
import pytz

In [33]:
model = pickle.load(open("models/4_rf_3.1f.pkl","rb"))

/Users/nikita_voitishyn/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/nikita_voitishyn/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [51]:
# predict for each region with merging and matrix inside

cities = ['Крим','Вінниччина','Волинь','Дніпропетровщина','Донеччина','Житомирщина','Закарпаття', 'Запоріжжя',
          'Івано-Франківщина','Київщина','Кіровоградщина','Луганщина','Львівщина','Миколаївщина','Одещина','Полтавщина',
          'Рівненщина','Сумщина','Тернопільщина','Харківщина','Херсонщина','Хмельниччина','Черкащина',
          'Буковина','Чернігівщина']

date = datetime.date.today()
result = {}
for city in cities:
    
    df_weather_final = get_weather.get_weather_for_12_hours(city,date)
    
    df_weather_final['key']=1
    df_isw_short['key']=1
    
    df_all = df_weather_final.merge(df_isw_short, how = 'left', left_on = 'key', right_on = 'key')
    to_drop=['key','date','date_tomorrow_datetime','keywords','report_text_lemm','datetime','preciptype','conditions',
       'icon', 'stations', 'source','sunrise','sunset']
    df_weather_matrix_v1 = df_all.drop(to_drop, axis = 1)
    
    cv_vector_model = cv.transform(df_all['report_text_lemm'].values.astype('U'))
    
    TF_IDF_MODEL = tfidf.transform(cv_vector_model)
    
    df_weather_matrix_v1_csr = scipy.sparse.csr_matrix(df_weather_matrix_v1)
#     df_all_data_csr = scipy.sparse.hstack((df_weather_matrix_v1_csr, TF_IDF_MODEL), format='csr')
    
    
#     #predict 
#     predicted = model.predict(df_all_data_csr)
#     current_time = datetime.datetime.now()
#     hours = []
#     for i in range(12):
#         hour = date + datetime.timedelta(hours=i)
#         hour_rounded = hour.replace(minute=0, second=0, microsecond=0)
#         hours.append(hour_rounded.strftime('%Y-%m-%d %H:%M'))

#     result[city] = dict(zip(hours, predicted))


Weather data for the 
region Крим; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Вінниччина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready


/var/folders/5y/jfstwx3n6x5bg_8mdp2wq81w0000gn/T/ipykernel_84184/3468653587.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isw_short['key']=1


KeyError: "['sunrise' 'sunset'] not found in axis"

In [49]:
to_drop=['key','date','date_tomorrow_datetime','keywords','report_text_lemm','datetime','preciptype','conditions',
       'icon', 'stations', 'source','sunrise','sunset']
df_weather_matrix_v1 = df_all.drop(to_drop, axis = 1)

In [50]:
df_weather_matrix_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetimeEpoch   24 non-null     int64  
 1   temp            24 non-null     float64
 2   feelslike       24 non-null     float64
 3   humidity        24 non-null     float64
 4   dew             24 non-null     float64
 5   precip          24 non-null     float64
 6   precipprob      24 non-null     float64
 7   snow            24 non-null     float64
 8   snowdepth       24 non-null     float64
 9   windgust        24 non-null     float64
 10  windspeed       24 non-null     float64
 11  winddir         24 non-null     float64
 12  pressure        24 non-null     float64
 13  visibility      24 non-null     float64
 14  cloudcover      24 non-null     float64
 15  solarradiation  24 non-null     float64
 16  solarenergy     15 non-null     float64
 17  uvindex         24 non-null     float

In [ ]:
df_weather_matrix_v1.values

In [ ]:
df_weather_matrix_v1.columns

In [47]:
df_weather_matrix_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetimeEpoch   24 non-null     int64  
 1   temp            24 non-null     float64
 2   feelslike       24 non-null     float64
 3   humidity        24 non-null     float64
 4   dew             24 non-null     float64
 5   precip          24 non-null     float64
 6   precipprob      24 non-null     float64
 7   snow            24 non-null     float64
 8   snowdepth       24 non-null     float64
 9   windgust        24 non-null     float64
 10  windspeed       24 non-null     float64
 11  winddir         24 non-null     float64
 12  pressure        24 non-null     float64
 13  visibility      24 non-null     float64
 14  cloudcover      24 non-null     float64
 15  solarradiation  24 non-null     float64
 16  solarenergy     15 non-null     float64
 17  uvindex         24 non-null     float

In [48]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetime                24 non-null     object        
 1   datetimeEpoch           24 non-null     int64         
 2   temp                    24 non-null     float64       
 3   feelslike               24 non-null     float64       
 4   humidity                24 non-null     float64       
 5   dew                     24 non-null     float64       
 6   precip                  24 non-null     float64       
 7   precipprob              24 non-null     float64       
 8   snow                    24 non-null     float64       
 9   snowdepth               24 non-null     float64       
 10  preciptype              0 non-null      object        
 11  windgust                24 non-null     float64       
 12  windspeed               24 non-null     float64     

In [ ]:
df_weather_matrix_v1.shape

### PREDICT

In [ ]:
merged_df.info()

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
print(data_merged.columns)

for column in data_merged.columns:
    print(column)

In [ ]:
ʼ